In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [74]:
##########################################
# --- Agente 1: Buscador de Preços --- #
##########################################

def agente_buscador(medicamento):

  buscador = Agent(
      name = "agente_buscador",
      # model = "gemini-2.0-flash",
      model = "gemini-2.5-flash-preview-04-17",
      description = "Agente que busca preços no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar os melhores preços e ofertas do medicamento abaixo.
      Os melhores preços seriam os com menor valor, tendo desconto ou não.
      Retorne no máximo 5 melhores preços, incluindo preços com descontos.

      Retorne na seguinte estrutura:
      Farmácia: Nome da Farmácia
      Medicamento: Nome
      Descrição: Descrição do medicamento
      Valor: Apenas o valor do medicamento encontrado na busca
      Link: Link de acesso ao site da farmacia, formate ele para um formato https valido
      """
  )

  entrada_do_agente_buscador = f"Medicamento: {medicamento}"

  resultado = call_agent(buscador, entrada_do_agente_buscador)
  return resultado

In [ ]:
print("Iniciando o Sistema de Busca de Medicamentos com Melhores Preços")

# --- Obter o Tópico do Usuário ---
medicamento = input("Por favor, digite o medicamento que você tem interesse: ")

# Inserir lógica do sistema de agentes ################################################

if not medicamento:
  print("É necessário informar pelo menos 1 medicamento")
else:
  print(f"Maravilha! Vamos buscar os melhores valores para {medicamento}.")

  resultado_pesquisa = agente_buscador(medicamento)
  print("\n------ Resultado da Busca ------\n")
  print(resultado_pesquisa)
  print("----------------------------------------------------")
